In [ ]:
#Dieses Python File dient zur Ermittlung des Impacts der einzelnen Engpassknoten auf die Zinsstrukturkurve
#Durch die manuelle Abwandlung der Werte, welche aus dem trainierten Modell übernommen werden, können neue 
#Zinsstrukturkurven erzeugt werden. Diese werden in Excel gespeichert und können danach mit der originalen 
#Zinsstrukturkurve verglichen werden

#Input, welcher benötigt wird: exportFamaBliss_final30.csv, exportFamaBlisstimeonly30.csv, 
#exportFamaBlissyieldsonly30.csv, mattimes3.csv, rownum3.csv, Encoder und Decoder

#Output: kurvexxx.xlsx

In [1]:
import csv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

In [2]:
decoder = keras.models.load_model("decoder0")
encoder = keras.models.load_model("encoder0")

In [3]:
# csv file name
filename = 'exportFamaBliss_final30.csv'
 
# initializing the titles and rows list
fields = []
x = []

with open(filename, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        x.append(row)

y = []
filename2 = 'exportFamaBlissyieldsonly30.csv'

with open(filename2, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        y.append(row)   
        
ti = []
filename3 = 'exportFamaBlisstimeonly30.csv'

with open(filename3, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        ti.append(row)   
        
#print(wichtig[0])
#len(wichtig)
#print(x[0])

for i in range(len(x)):
    x[i] = np.array_split(x[i], 2)
    
x1 = pad_sequences(x,maxlen=None,dtype='float64')
y1 = pad_sequences(y,maxlen=None,dtype='float64')
ti1 = pad_sequences(ti,maxlen=None,dtype='float64')

x_train, x_test, y_train, y_test = train_test_split(ti1, y1, test_size=0.15, random_state=1)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

array111 = encoder.predict([y1,ti1])

### Änderung der Engpassknoten, um Veränderungen zu verdeutlichen
### Untersuchung des Stichtages 4601

array111 = array111[[4600]]

### Dieser Wert muss angepasst werden:
veraenderung = 0.2
array111[0][2] = array111[0][2] - veraenderung

In [4]:
tiexport = []
### Seq(0.20 bis 50 in 0.01 Abstand) ###
filename4 = 'mattimes3.csv'

with open(filename4, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        tiexport.append(row)   
        
        
rownums = []
filename5 = 'rownum3.csv'

with open(filename5, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        rownums.append(row)   
        
rownums = pad_sequences(rownums,maxlen=None,dtype='float64')

In [5]:
arrayfinal = []
tiexport1 = pad_sequences(tiexport,maxlen=None,dtype='float64')

for i in range(len(array111)):
    if i==0:
        a = 0
        b = int(rownums[0][0])
        tiexport1_new = tiexport1[a:b]
        input1 = []
        for _ in range(len(tiexport1_new)):
            input1.append(array111[i])
        
        input1 = pad_sequences(input1,maxlen=None,dtype='float64')
        array222 = decoder.predict([input1,tiexport1_new])
        
        arrayfinal.append(array222)
    else:
        a = int(sum(rownums[0:i])[0])
        b = int(sum(rownums[0:(i+1)])[0])

        tiexport1_new = tiexport1[a:b]
        input1 = []
        for _ in range(len(tiexport1_new)):
            input1.append(array111[i])
        
        input1 = pad_sequences(input1,maxlen=None,dtype='float64')
        array222 = decoder.predict([input1,tiexport1_new])
        
        arrayfinal.append(array222)
        

In [6]:
arraytestt = []
for i in range(len(arrayfinal)):
    for j in range(len(arrayfinal[i])):
        arraytestt.append(arrayfinal[i][j])
        
import pandas as pd

arrayfin = pd.DataFrame(arraytestt)

### Output der veränderten Zinsstrukturkurve
arrayfin.to_excel (r'kurvexxx.xlsx', index = False, header=True)

In [7]:
#array111 = encoder.predict([y_test,x_test])

#import pandas as pd

#df = pd.DataFrame(array111, columns = ['x1','x2','x3'])
#df.to_excel (r'export_testdata.xlsx', index = False, header=True)
